In [1]:
#Import required packages
import requests
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

import time
import os
import pickle

#### The following Cell contains a function that will scrape bitcoin talk forum using selenium webdriver for links to individual posts that users generate to discuss bitcoin news

In [36]:
#Set chromedriver executable path. 
chromedriver = "C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe" # path to the chromedriver executable, set accordingly
os.environ["webdriver.chrome.driver"] = chromedriver

#Define function to extract forum discussion links. 
def get_bitcoin_news_links(url, start_date, end_date):
    """
    A function utilizing selenium to scrape bitcoin 15-min interval price history from bitcoin char
    """
    driver = webdriver.Chrome(chromedriver)
    df = pd.DataFrame(columns=['timestamp','open','high','low','close','volume_btc','volume_currency','weighted price'])
    print('Scraping Bitcoin News Links\n')

    #Establish link
    bitcoinnews_url=url

    #Utilize selenium drive to launch link
    driver.get(bitcoinnews_url)
#     driver.fullscreen_window()
    driver.find_element_by_xpath("//*[@id='c']").click()
    
    search_bar = driver.find_element_by_id("i")
    for option in search_bar.find_elements_by_tag_name('option'):
        if option.text == '15-min':
            option.click() 
            break
    
    start_date_input = driver.find_element_by_id("s")
    start_date_input.clear()
    start_date_input.click()
    start_date_input.send_keys('2019-05-01')
    start_date_input.send_keys(Keys.RETURN)
    
    end_date_input = driver.find_element_by_id("e")
    end_date_input.clear()
    end_date_input.click()
    end_date_input.send_keys('2019-05-30')
    end_date_input.send_keys(Keys.RETURN)
   
    
    element = driver.find_element_by_xpath('//*[@id="content_chart"]/div/div[2]/a')
    driver.execute_script("return arguments[0].scrollIntoView(0, document.documentElement.scrollHeight-10);", element)
    
    #element = driver.find_element_by_xpath('//*[@id="content_chart"]/div/div[2]/a')
    #element.find_elements_by_xpath('//*[@id="content_chart"]/div/div[2]/a')
    #element.click()

#     for row in rows:
#         link = row.find_element(By.TAG_NAME, "a")
#         a_elements.append(link.get_attribute('href'))

        
    print('Scraping Done!\n')
    
    return None

In [37]:
#Define site URL for scraping
url="https://bitcoincharts.com/charts/bitstampUSD#rg60ztgSzm1g10zm2g25zv"

#Scrap News Article Posts. Only run once then comment out, will take approximately 5-10min
forum_subject_links = get_bitcoin_news_links(url,'t','t')

Scraping Bitcoin News Links

Scraping Done!



In [35]:
#Set chromedriver executable path. 
chromedriver = "C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

#Define function to extract walkability, transit, and bike indexes
def get_info_and_scores(url, neighborhood_list):
    driver = webdriver.Chrome(chromedriver)
    df = pd.DataFrame(columns=['neighborhood', 'walk_score_of_100','transit_score_of_100','bike_score_of_100','median_sale_price_$'])
    print('Scraping Neighborhoods from Redfin\n')
    
    for neighborhood in neighborhood_list:
        print('Scraping ' + neighborhood)
        redfin_url=url
        driver.get(redfin_url)
        
        search_bar = driver.find_element_by_id("search-box-input")
        search_bar.send_keys(neighborhood + ', San Francisco, CA')
        search_bar.send_keys(Keys.RETURN)
        
        
        
        #Wait 10 seconds for page to load
        WebDriverWait(driver, 10).until(EC.url_changes(redfin_url))
        
        try:
            #Find div containing walk-scores class and median sale price 
            content_walk = driver.find_element_by_xpath("//div[contains(@class, 'walk-score')]")
            content_med_sale_price = driver.find_element_by_xpath("//div[contains(@class, 'trends')]//li[5]//div//span[2]//span")
            
            #convert div html to text
            content_walk_text =content_walk.text
            content_med_sale_text = content_med_sale_price.text
            
            #Extract out of 100 scores from div
            info_scores = [int(s) for s in content_walk_text.split() if s.isdigit()]
            info_scores = [info_scores[0],info_scores[2],info_scores[4]]
            
            #Add neighborhood and median sale price to info_list
            info_scores.insert(0,neighborhood)
            info_scores.insert(len(info_scores),content_med_sale_text)
        
            #add to walkability dataframe
            df.loc[-1] = info_scores  # adding a row
            df.index = df.index + 1  # shifting index
            df = df.sort_index()
        
        except NoSuchElementException:
            filler=[neighborhood,np.NaN,np.NaN,np.NaN,np.NaN]
            df.loc[-1] = filler  # adding a row
            df.index = df.index + 1  # shifting index
            df = df.sort_index()
    
    print('Scraping Done!')
            
    return df

#Define site URL for scraping
url="https://www.redfin.com/"

#Define neighborhood list as available in the SF data
neighborhood_list = ['Western Addition', 'Bernal Heights', 'Haight Ashbury', 'Mission',\
       'Potrero Hill', 'Civic Center / Van Ness', 'Castro','Upper Market',\
       'Inner Sunset', 'South of Market', 'Noe Valley', 'Outer Richmond',\
       'Presidio Heights', 'Nob Hill', 'Ocean View Terrace', 'Pacific Heights',\
       'Financial District', 'Twin Peaks', 'Russian Hill', 'Outer Sunset',\
       'North Beach', 'Glen Park', 'Marina Distric', 'Inner Richmond',\
       'Excelsior', 'Seacliff', 'Chinatown', 'Bayview', 'Diamond Heights',\
       'West of Twin Peaks', 'Outer Mission', 'Parkside', 'Lakeshore',\
       'Crocker Amazon', 'Golden Gate Park', 'Visitacion Valley','Presidio Heights']
